In [ ]:
# Imports
import subprocess
import pathlib
import os
import logging
import sys
from datetime import datetime

In [ ]:
from utils import find_compound_name_by_cid

In [ ]:
# Logging setup
time_start = datetime.now()
log_filename = f"{time_start.year}-{str(time_start.month).zfill(2)}-{str(time_start.day).zfill(2)}_{str(time_start.hour).zfill(2)}_{str(time_start.minute).zfill(2)}_{str(time_start.second).zfill(2)}"

os.makedirs("./logs/", exist_ok = True)

with open(f"./logs/{log_filename}.log", 'w', encoding = 'utf-8') as f:
    f.truncate(0)

logging.basicConfig(level = 'INFO', format = '[%(asctime)s] [%(thread)d - %(threadName)s] [%(levelname)s] %(message)s', encoding = 'utf-8',
                    handlers = [
                        logging.FileHandler(f"./logs/{log_filename}.log", encoding = 'utf-8'),
                        logging.StreamHandler(stream=sys.stdout)
                    ])

In [ ]:
os.makedirs('./ligands', exist_ok=True)
os.makedirs('./receptors', exist_ok=True)
os.makedirs('./vina', exist_ok=True)

In [ ]:
# Configs
vina_path = pathlib.Path('C:/Users/Administrator/Desktop/Programs/AutoDock Vina/vina.exe')
ligands_path = pathlib.Path('./ligands')
receptors_path = pathlib.Path('./receptors')

class VinaParams:
    def __init__(self, center_x, center_y, center_z, size_x, size_y, size_z, exhaustiveness = 8, num_modes = 9, cpu = 16):
        self.center_x = center_x
        self.center_y = center_y
        self.center_z = center_z
        self.size_x = size_x
        self.size_y = size_y
        self.size_z = size_z
        self.cpu = cpu
        
        # how thorough the search will be. Higher values increase the search thoroughness, 
        # but also increase the time it takes to complete the docking
        # The default value is 8
        self.exhaustiveness = exhaustiveness
        
        # how many binding modes will be generated
        # The default value is 9
        self.num_modes = num_modes

paramSetDict = {
    
}

logging.info(f'Initialization Complete')

In [ ]:
receptors = list(receptors_path.glob('*.pdbqt'))
ligands = list(ligands_path.glob('*.pdbqt'))
logging.info(f'Found {len(receptors)} Receptors, {len(ligands)} Ligands')

# get all receptor files
for receptor in receptors:
    receptor_name = receptor.stem
    logging.info(f'Receptor: {receptor_name}')

    if receptor_name not in paramSetDict:
        logging.error(f"No defined parameters found for receptor: {receptor_name}")
        continue
    else:
        paramSet = paramSetDict.get(receptor_name)
        
    # get all ligand files
    for ligand in ligands:
        ligand_cid = ligand.stem
        ligand_name = find_compound_name_by_cid(ligand_cid)
        logging.info(f'Ligand: {ligand_name}')
        receptor_file = receptors_path/receptor
        ligand_file = ligands_path/ligand
        

        
        # run vina
        command = [
            str(vina_path),
            '--receptor', str(receptor_file),
            '--ligand', str(ligand_file),
            '--center_x', str(paramSet.center_x),
            '--center_y', str(paramSet.center_y),
            '--center_z', str(paramSet.center_z),
            '--size_x', str(paramSet.size_x),
            '--size_y', str(paramSet.size_y),
            '--size_z', str(paramSet.size_z),
            '--exhaustiveness', str(paramSet.exhaustiveness),
            '--num_modes', str(paramSet.num_modes),
            '--cpu', str(paramSet.cpu),
            '--out', f'./output/{receptor_name}/{ligand_name}[{ligand_cid}].pdbqt'  # Output file for docking results
            # '--log', f'./output/{receptor_name}/{ligand_name}[{ligand_cid}].log'    # Log file
        ]
        try:
            result = subprocess.run(command, cwd="./", check=True, capture_output=True, text=True)
            
            stdout = result.stdout
            
            # TODO: Post-process the output
            
            stderr = result.stderr
            logging.info(f"Completed: {ligand_name} - {receptor_name}")
            
            if stderr:
                logging.warning(f"Error Output: {stderr}")
        except subprocess.CalledProcessError as e:
            logging.error(f"Error while running Vina: {e}")
            continue
        except Exception as e:
            logging.error(f"Unexpected Error: {e}")
            continue